In [24]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from textblob.sentiments import PatternAnalyzer
import regex as re

In [25]:
BH_tweets_during_war = pd.read_csv('dane/BH_tweets_during_war.csv', lineterminator='\n')

In [26]:
BH_tweets_during_war.describe()

,id,conversation_id,timezone,user_id,replies_count,retweets_count,likes_count,video,near,geo,source,user_rt_id,user_rt,retweet_id,retweet_date,translate,trans_src,trans_dest
count,3.235600e+04,3.235600e+04,32356.0,3.235600e+04,32356.000000,32356.000000,32356.000000,32356.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1.477051e+18,1.476570e+18,200.0,5.155764e+17,0.720361,1.477068,7.840710,0.355359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2.264122e+16,2.662017e+16,0.0,5.993913e+17,11.520739,46.841371,202.211277,0.478629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.432859e+18,1.875374e+17,200.0,5.440000e+02,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.457145e+18,1.457075e+18,200.0,2.241061e+08,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.480557e+18,1.480316e+18,200.0,2.940557e+09,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.497595e+18,1.497592e+18,200.0,1.161648e+18,0.000000,0.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,1.509678e+18,1.509678e+18,200.0,1.508394e+18,1088.000000,6621.000000,32103.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
BH_tweets_during_war.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32356 entries, 0 to 32355
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               32356 non-null  int64  
 1   conversation_id  32356 non-null  int64  
 2   created_at       32356 non-null  object 
 3   date             32356 non-null  object 
 4   time             32356 non-null  object 
 5   timezone         32356 non-null  int64  
 6   user_id          32356 non-null  int64  
 7   username         32356 non-null  object 
 8   name             32356 non-null  object 
 9   place            520 non-null    object 
 10  tweet            32356 non-null  object 
 11  language         32356 non-null  object 
 12  mentions         32356 non-null  object 
 13  urls             32356 non-null  object 
 14  photos           32356 non-null  object 
 15  replies_count    32356 non-null  int64  
 16  retweets_count   32356 non-null  int64  
 17  likes_count 

In [28]:
BH_tweets_during_war.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1509677705280868360,1509677686700089348,2022-04-01 01:43:08 CEST,2022-04-01,01:43:08,200,1093447620893331457,peregreine,Peregrine Trader,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1509675515128471555,1509675515128471555,2022-04-01 01:34:25 CEST,2022-04-01,01:34:25,200,162060435,traceyryniec,Tracey Ryniec,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1509671718545235969,1509671718545235969,2022-04-01 01:19:20 CEST,2022-04-01,01:19:20,200,332364947,simonsalhanyre,BHHS Simon & Salhany,"{'type': 'Point', 'coordinates': [41.0379043, ...",...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1509670638801334272,1509670638801334272,2022-04-01 01:15:03 CEST,2022-04-01,01:15:03,200,2738590284,kevinhillre,Kevin Hill,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1509667550791094281,1509667550791094281,2022-04-01 01:02:47 CEST,2022-04-01,01:02:47,200,827212847012773888,diamondrafargo,Diamond Realty Assoc,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [29]:
#Add date format and sorted by dates
BH_tweets_during_war['date'] = pd.to_datetime(BH_tweets_during_war['date'], format = '%Y-%m-%d')
BH_tweets_during_war = BH_tweets_during_war.sort_values(by='date')
BH_tweets_during_war = BH_tweets_during_war.reset_index(drop=True)
BH_tweets_during_war

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1432859098719084546,1432859098719084546,2021-09-01 02:13:25 CEST,2021-09-01,02:13:25,200,786038665625554944,stocktiprobot,Stock Advicebot 2000,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1433120314892951560,1433120314892951560,2021-09-01 19:31:23 CEST,2021-09-01,19:31:23,200,72845155,kimvanhal,Kimberly Van Hal,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1433119271337578505,1433109408200990724,2021-09-01 19:27:15 CEST,2021-09-01,19:27:15,200,3662626821,yolanda240693,YOLANDA GUERRERO,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'josep_bagant', 'name': 'Jose...",NaN,NaN,NaN,NaN
3,1433119247555653633,1433119247555653633,2021-09-01 19:27:09 CEST,2021-09-01,19:27:09,200,3044925008,bhhscpchulavsta,BHHSCPChulaVista,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1433116584328024074,1433116584328024074,2021-09-01 19:16:34 CEST,2021-09-01,19:16:34,200,409521300,kellydunnrynes,North Shore Broker,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32351,1509654314792464395,1509654314792464395,2022-04-01 00:10:11 CEST,2022-04-01,00:10:11,200,732736306569170944,francie_grieb,Francie grieb,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
32352,1509652247260635136,1509652247260635136,2022-04-01 00:01:58 CEST,2022-04-01,00:01:58,200,2344133624,jana_realtor,Jana Xu,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
32353,1509651973217341445,1509651973217341445,2022-04-01 00:00:53 CEST,2022-04-01,00:00:53,200,316914016,ilvre,LasVegasRealEstate,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
32354,1509663472807727106,1509663472807727106,2022-04-01 00:46:34 CEST,2022-04-01,00:46:34,200,47431775,lhamptonteam,Louise Hampton Team,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [30]:
#Remove tweets writeen in other language than english
BH_tweets_during_war_clean = BH_tweets_during_war[BH_tweets_during_war.language == 'en']
BH_tweets_during_war_clean = BH_tweets_during_war_clean.reset_index(drop=True)
BH_tweets_during_war_clean.shape

(29079, 36)

In [31]:
#Remove columns without any data
BH_tweets_during_war_clean = BH_tweets_during_war_clean.dropna(thresh = 1, axis = 1)

In [32]:
#3782 records and 10 columns were removed
BH_tweets_during_war_clean.describe()

,id,conversation_id,timezone,user_id,replies_count,retweets_count,likes_count,video
count,2.907900e+04,2.907900e+04,29079.0,2.907900e+04,29079.000000,29079.000000,29079.000000,29079.000000
mean,1.476758e+18,1.476277e+18,200.0,5.094413e+17,0.719832,1.487053,7.588707,0.369786
std,2.269436e+16,2.668803e+16,0.0,5.976456e+17,12.103783,49.332784,212.588995,0.482755
min,1.432859e+18,1.875374e+17,200.0,5.440000e+02,0.000000,0.000000,0.000000,0.000000
25%,1.457022e+18,1.456991e+18,200.0,2.165727e+08,0.000000,0.000000,0.000000,0.000000
50%,1.479914e+18,1.479843e+18,200.0,2.920146e+09,0.000000,0.000000,0.000000,0.000000
75%,1.497583e+18,1.497581e+18,200.0,1.157997e+18,0.000000,0.000000,1.000000,1.000000
max,1.509678e+18,1.509678e+18,200.0,1.508394e+18,1088.000000,6621.000000,32103.000000,1.000000


In [33]:
BH_tweets_during_war_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29079 entries, 0 to 29078
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               29079 non-null  int64         
 1   conversation_id  29079 non-null  int64         
 2   created_at       29079 non-null  object        
 3   date             29079 non-null  datetime64[ns]
 4   time             29079 non-null  object        
 5   timezone         29079 non-null  int64         
 6   user_id          29079 non-null  int64         
 7   username         29079 non-null  object        
 8   name             29079 non-null  object        
 9   place            519 non-null    object        
 10  tweet            29079 non-null  object        
 11  language         29079 non-null  object        
 12  mentions         29079 non-null  object        
 13  urls             29079 non-null  object        
 14  photos           29079 non-null  objec

In [34]:
BH_tweets_during_war_clean.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to
0,1432859098719084546,1432859098719084546,2021-09-01 02:13:25 CEST,2021-09-01,02:13:25,200,786038665625554944,stocktiprobot,Stock Advicebot 2000,NaN,...,0,0,['wealth'],['dlr'],https://twitter.com/StockTipRobot/status/14328...,False,NaN,0,NaN,[]
1,1433120314892951560,1433120314892951560,2021-09-01 19:31:23 CEST,2021-09-01,19:31:23,200,72845155,kimvanhal,Kimberly Van Hal,NaN,...,0,0,[],[],https://twitter.com/kimvanhal/status/143312031...,False,NaN,1,https://pbs.twimg.com/media/E-N3VJ6WEAEqYKZ.jpg,[]
2,1433119247555653633,1433119247555653633,2021-09-01 19:27:09 CEST,2021-09-01,19:27:09,200,3044925008,bhhscpchulavsta,BHHSCPChulaVista,NaN,...,0,0,"['foreverbrand', 'reputationmatters', 'joinus'...",[],https://twitter.com/BHHSCPChulaVsta/status/143...,False,NaN,1,https://pbs.twimg.com/media/E-N2VvoVcAEnisi.jpg,[]
3,1433116584328024074,1433116584328024074,2021-09-01 19:16:34 CEST,2021-09-01,19:16:34,200,409521300,kellydunnrynes,North Shore Broker,NaN,...,0,0,['chicagotothenorthshore'],[],https://twitter.com/KellyDunnRynes/status/1433...,False,NaN,0,NaN,[]
4,1433114614229659649,1433114614229659649,2021-09-01 19:08:44 CEST,2021-09-01,19:08:44,200,15691063,lanceberwald,Lance Berwald,"{'type': 'Point', 'coordinates': [44.91371, -9...",...,0,0,[],[],https://twitter.com/LanceBerwald/status/143311...,False,NaN,0,NaN,[]


In [35]:
#Check values of column place
BH_tweets_during_war_clean['place'].value_counts()

{'type': 'Point', 'coordinates': [28.16028, -82.38634]}          39
{'type': 'Point', 'coordinates': [44.6981658, -93.2177749]}      38
{'type': 'Point', 'coordinates': [33.7864851, -84.38743]}        23
{'type': 'Point', 'coordinates': [33.82291021, -116.5135278]}    20
{'type': 'Point', 'coordinates': [40.78806845, -74.47095514]}    16
                                                                 ..
{'type': 'Point', 'coordinates': [30.38838, -86.42039]}           1
{'type': 'Point', 'coordinates': [40.154911, -75.2685089]}        1
{'type': 'Point', 'coordinates': [43.0569411, -86.2233732]}       1
{'type': 'Point', 'coordinates': [39.9513708, -75.1723118]}       1
{'type': 'Point', 'coordinates': [33.78589024, -84.38724273]}     1
Name: place, Length: 118, dtype: int64

In [36]:
#Check values of column quote_url
BH_tweets_during_war_clean['quote_url'].value_counts()

https://twitter.com/RBReich/status/1435348653284237318            7
https://twitter.com/bitpanda/status/1485923043209879552           5
https://twitter.com/YahooFinance/status/1493996148624961542       4
https://twitter.com/Blockworks_/status/1494025302699384835        3
https://twitter.com/Dinesh_Sairam/status/1447239466255593483      3
                                                                 ..
https://twitter.com/_jonasschnelli_/status/1508518106259410947    1
https://twitter.com/Reuters_Health/status/1508768456509902856     1
https://twitter.com/JoeBiden/status/1505587220954206221           1
https://twitter.com/Anumuthuraayan/status/1429354777066958856     1
https://twitter.com/MayorJD/status/1467494571257077762            1
Name: quote_url, Length: 656, dtype: int64

In [37]:
#Counted quantity of links for each tweet
BH_tweets_during_war_clean['quote_url'] = BH_tweets_during_war_clean['quote_url'].str.count('https://')

#Replayce NaN by 0
BH_tweets_during_war_clean.fillna({'quote_url':0}, inplace=True)

#Changed data type from float64 to int64
BH_tweets_during_war_clean['quote_url'] = BH_tweets_during_war_clean['quote_url'].astype(np.int64)

In [38]:
#Replayce NaN to [] in thumbnail column
BH_tweets_during_war_clean.fillna({'thumbnail':'[]'}, inplace=True)

In [39]:
#There is no NaN except column place which will be removed due to low importance
BH_tweets_during_war_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29079 entries, 0 to 29078
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               29079 non-null  int64         
 1   conversation_id  29079 non-null  int64         
 2   created_at       29079 non-null  object        
 3   date             29079 non-null  datetime64[ns]
 4   time             29079 non-null  object        
 5   timezone         29079 non-null  int64         
 6   user_id          29079 non-null  int64         
 7   username         29079 non-null  object        
 8   name             29079 non-null  object        
 9   place            519 non-null    object        
 10  tweet            29079 non-null  object        
 11  language         29079 non-null  object        
 12  mentions         29079 non-null  object        
 13  urls             29079 non-null  object        
 14  photos           29079 non-null  objec

In [40]:
#Check duplicates
BH_tweets_during_war_clean[BH_tweets_during_war_clean.duplicated(subset = 'id')]

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to


In [41]:
#Column retweet consist of only False values
BH_tweets_during_war_clean['retweet'].value_counts()

False    29079
Name: retweet, dtype: int64

In [42]:
#Removed irrelevant columns
BH_tweets_during_war_clean = BH_tweets_during_war_clean.drop(['conversation_id','created_at', 'time', 
                                                              'timezone', 'language', 'retweet', 'link','place'], 
                                                             axis = 1)

In [43]:
#Counted number of elements stored as lists in columns
columns_lists = ['mentions', 'urls', 'photos', 'hashtags', 'cashtags', 'thumbnail']
for column in columns_lists:
    empty_list = BH_tweets_during_war_clean[column] == '[]'
    BH_tweets_during_war_clean[column] = BH_tweets_during_war_clean[column].str[1:-1].str.split(',')
    BH_tweets_during_war_clean[column] = BH_tweets_during_war_clean[column].str.len()
    BH_tweets_during_war_clean.loc[empty_list, column] = 0

In [44]:
#Counted number of elements in dictionaries stored in reply_to column
BH_tweets_during_war_clean['reply_to'] = BH_tweets_during_war_clean['reply_to'].str.count('{')

#Changed data type from float64 to int64
BH_tweets_during_war_clean['reply_to'] = BH_tweets_during_war_clean['reply_to'].astype(np.int64)

In [45]:
#Removed hastags, mentions, url and special signs from tweets
for i in range(BH_tweets_during_war_clean.shape[0]):
    BH_tweets_during_war_clean.loc[i, 'tweet'] = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",
                            BH_tweets_during_war_clean.loc[i, 'tweet']).split())

In [46]:
#Last day 2022-04-01 has only 14 tweets, it will be removed
BH_tweets_during_war_clean['date'].value_counts()

2022-03-21    812
2022-02-26    646
2022-02-17    496
2022-02-18    470
2022-03-17    436
             ... 
2021-12-12     59
2021-09-05     56
2021-10-02     55
2021-11-28     39
2022-04-01     14
Name: date, Length: 213, dtype: int64

In [47]:
BH_tweets_during_war_clean = BH_tweets_during_war_clean[BH_tweets_during_war_clean['date'] != '2022-04-01']
BH_tweets_during_war_clean['date'].value_counts()

2022-03-21    812
2022-02-26    646
2022-02-17    496
2022-02-18    470
2022-03-17    436
             ... 
2021-09-12     61
2021-12-12     59
2021-09-05     56
2021-10-02     55
2021-11-28     39
Name: date, Length: 212, dtype: int64

In [48]:
#Checked if there are tweets that contains only white spaces, numbers or special signs
BH_tweets_during_war_clean[~BH_tweets_during_war_clean["tweet"].str.contains('[A-Za-z]')]

,id,date,user_id,username,name,tweet,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,quote_url,video,thumbnail,reply_to


In [49]:
BH_tweets_during_war_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29065 entries, 0 to 29064
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              29065 non-null  int64         
 1   date            29065 non-null  datetime64[ns]
 2   user_id         29065 non-null  int64         
 3   username        29065 non-null  object        
 4   name            29065 non-null  object        
 5   tweet           29065 non-null  object        
 6   mentions        29065 non-null  int64         
 7   urls            29065 non-null  int64         
 8   photos          29065 non-null  int64         
 9   replies_count   29065 non-null  int64         
 10  retweets_count  29065 non-null  int64         
 11  likes_count     29065 non-null  int64         
 12  hashtags        29065 non-null  int64         
 13  cashtags        29065 non-null  int64         
 14  quote_url       29065 non-null  int64         
 15  vi

In [50]:
#Save cleaned data in csv format
BH_tweets_during_war_clean.to_csv('dane/BH_tweets_during_war_clean.csv', index=False)

In [51]:
#Create dataframe for textanalysis for each day
BH_tweets_during_war_textanalysis = pd.date_range(BH_tweets_during_war_clean['date'].min(),
                                                       BH_tweets_during_war_clean['date'].max(),freq='d')
BH_tweets_during_war_textanalysis = pd.DataFrame(BH_tweets_during_war_textanalysis, columns = ['date'])
column_names = ['Short negative factual tweets', 'Short negative personal tweets', 'Short positive factual tweets',
                'Short positive personal tweets', 'Short neutral factual tweets', 'Short neutral personal tweets',
                'Long negative factual tweets', 'Long negative personal tweets', 'Long positive factual tweets',
                'Long positive personal tweets', 'Long neutral factual tweets', 'Long neutral personal tweets',
                'Total amount of tweets']
for column in column_names:
    BH_tweets_during_war_textanalysis[column] = 0
BH_tweets_during_war_textanalysis

,date,Short negative factual tweets,Short negative personal tweets,Short positive factual tweets,Short positive personal tweets,Short neutral factual tweets,Short neutral personal tweets,Long negative factual tweets,Long negative personal tweets,Long positive factual tweets,Long positive personal tweets,Long neutral factual tweets,Long neutral personal tweets,Total amount of tweets
0,2021-09-01,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2021-09-02,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2021-09-03,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2021-09-04,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2021-09-05,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,2022-03-27,0,0,0,0,0,0,0,0,0,0,0,0,0
208,2022-03-28,0,0,0,0,0,0,0,0,0,0,0,0,0
209,2022-03-29,0,0,0,0,0,0,0,0,0,0,0,0,0
210,2022-03-30,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
#Analysis of opinion and number of words using the textblob package
for i in range(BH_tweets_during_war_clean.shape[0]):
    blob = TextBlob(BH_tweets_during_war_clean.loc[i, 'tweet'], analyzer=PatternAnalyzer())
    if len(blob.words) < 30:
        if blob.sentiment[0] < 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Short negative factual tweets'] += 1
            else:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Short negative personal tweets'] += 1
        elif blob.sentiment[0] > 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Short positive factual tweets'] += 1
            else:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Short positive personal tweets'] += 1
        else:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Short neutral factual tweets'] += 1
            else:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Short neutral personal tweets'] += 1
    else:
        if blob.sentiment[0] < 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Long negative factual tweets'] += 1
            else:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Long negative personal tweets'] += 1
        elif blob.sentiment[0] > 0:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Long positive factual tweets'] += 1
            else:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Long positive personal tweets'] += 1
        else:
            if blob.sentiment[1] < 0.5:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Long neutral factual tweets'] += 1
            else:
                BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Long neutral personal tweets'] += 1
    BH_tweets_during_war_textanalysis.loc[BH_tweets_during_war_textanalysis[BH_tweets_during_war_textanalysis['date'] == BH_tweets_during_war_clean.loc[i, 'date']].index.values[0], 
                                                               'Total amount of tweets'] += 1

In [53]:
BH_tweets_during_war_textanalysis

,date,Short negative factual tweets,Short negative personal tweets,Short positive factual tweets,Short positive personal tweets,Short neutral factual tweets,Short neutral personal tweets,Long negative factual tweets,Long negative personal tweets,Long positive factual tweets,Long positive personal tweets,Long neutral factual tweets,Long neutral personal tweets,Total amount of tweets
0,2021-09-01,1,2,11,15,28,0,1,0,13,18,7,0,96
1,2021-09-02,3,1,30,15,23,1,2,2,20,18,4,2,121
2,2021-09-03,4,1,21,12,43,0,2,0,17,9,5,4,118
3,2021-09-04,3,1,18,12,22,0,5,0,16,13,6,0,96
4,2021-09-05,0,2,12,6,14,0,0,1,6,11,3,1,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,2022-03-27,4,3,14,59,25,1,4,3,20,14,3,1,151
208,2022-03-28,0,2,21,31,37,3,3,1,19,24,6,0,147
209,2022-03-29,3,2,19,23,40,1,3,0,23,22,3,2,141
210,2022-03-30,4,4,33,22,33,0,4,1,17,16,8,0,142


In [54]:
BH_tweets_during_war_textanalysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   date                            212 non-null    datetime64[ns]
 1   Short negative factual tweets   212 non-null    int64         
 2   Short negative personal tweets  212 non-null    int64         
 3   Short positive factual tweets   212 non-null    int64         
 4   Short positive personal tweets  212 non-null    int64         
 5   Short neutral factual tweets    212 non-null    int64         
 6   Short neutral personal tweets   212 non-null    int64         
 7   Long negative factual tweets    212 non-null    int64         
 8   Long negative personal tweets   212 non-null    int64         
 9   Long positive factual tweets    212 non-null    int64         
 10  Long positive personal tweets   212 non-null    int64         
 11  Long n

In [55]:
#Save text analysis data in csv format
BH_tweets_during_war_textanalysis.to_csv('dane/BH_tweets_during_war_textanalysis.csv', index=False)